In [58]:
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from fealpy.decorator import cartesian
from fealpy.mesh import TriangleMesh
from fealpy.functionspace import LagrangeFiniteElementSpace
from fealpy.boundarycondition import DirichletBC

class pde():
    def __init__(self, mu=1, lam=1):
        self.mu  = mu
        self.lam = lam
        
    def domain(self):
        return [0, 1, 0, 1]
    
    def init_mesh(self, n=1, meshtype='tri'):
        node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
        cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
        mesh = TriangleMesh(node, cell)
        mesh.uniform_refine(n)
        return mesh
    
    @cartesian
    def source(self, p):
        x = p[..., 0]
        y = p[..., 1]
        
        var = np.zeros(p.shape, dtype=np.float64)
        var[..., 0] = -((12 * x**2 - 12 * x + 2) * (4 * y**3 - 6 * y**2 + 2 * y) 
                        + (x - x**2)**2 * (24 * y - 12))
        var[..., 1] = ((24 * x - 12) * (y - y**2)**2  
                        +(4 * x**3 - 6 * x**2 + 2 * x) * (12 * y**2 -12 * y + 2))
        return var
    
    def dirichlet(self, p):
        var = np.zeros_like(p)
        return var
    
    def is_dirichlet_boundary(self, p):
        x = p[..., 0]
        y = p[..., 1]
        flag = np.max(np.abs(x)) < 1e-13 or np.max(np.abs(y)) < 1e-13
    
pde = pde()
mesh = pde.init_mesh(2)
NN = mesh.number_of_nodes()
NC = mesh.number_of_cells()
#[NC,2] 剖分点及其编号(下标)
node = mesh.entity('node') 
#[NC,3] 剖分区间及其端点编号
cell = mesh.entity('cell') 
#[NC] 每个单元的面积
cm = mesh.entity_measure()

#[NC,3,2] 每个单元上基函数(三个)对x、y的偏导数
glam = mesh.grad_lambda()
GLAM = np.broadcast_to(glam[:, :, None, :], shape=[NC,3,2,2])

#[NC,3,3] 单元刚度矩阵第一部分
S = np.einsum("cnij, cmij, c->cnm", GLAM, GLAM, cm)
glam_sum = np.einsum("cni->cn", glam)
#[NC,3,3] 单元刚度矩阵第二部分
M = np.einsum("cn, cm->cnm", glam_sum, glam_sum)

I = np.broadcast_to(cell[:, :, None], shape=S.shape)
J = np.broadcast_to(cell[:, None, :], shape=S.shape)

S = csr_matrix((S.flat, (I.flat, J.flat)), shape=(NN,NN))
M = csr_matrix((M.flat, (I.flat, J.flat)), shape=(NN,NN))
A = S + M

In [59]:
#fig = plt.figure()
#ax = fig.gca()
#mesh.add_plot(ax)
#mesh.find_node(ax,showindex=True)
#mesh.find_cell(ax,showindex=True)

#print("NN= ", NN)
#print("NC= ", NC)
#print("glam= \n", glam)
#print("GLAM= ", GLAM)
#print("S= ", S.toarray())
#print("glam_sum= \n", glam_sum)
#print("M= ", M.toarray())
#print("node= ", node)
#print("cell= ", cell)
#print("I= ", I)
#print("J= ", J)

In [60]:
from fealpy.quadrature import TriangleQuadrature

NQ = 2
Q = NQ * (NQ + 1) / 2
qf = TriangleQuadrature(NQ)

#bcs [Q,3]; WS[Q,]
bcs,ws = qf.get_quadrature_points_and_weights()
#phi [Q,3,2]
phi = np.broadcast_to(bcs[:, :, None], shape=[int(Q),3,2])

#ps [Q,NC,2]
ps = np.einsum('qi, cim->qcm', bcs, node[cell])
#val [Q,NC,2]
val = pde.source(ps)
bb = np.einsum('q, qcj, qij, c->ci', ws, val, phi, cm)

F = np.zeros(NN)
np.add.at(F, cell, bb)

isBdNode = mesh.ds.boundary_node_flag()
isInterNode = ~isBdNode

uh = np.zeros((NN), dtype=np.float64)
uh[isInterNode] = spsolve(A[:, isInterNode][isInterNode], F[isInterNode])

In [61]:
#print("bcs= ", bcs)
#print("phi= ", phi)
#print("ws= ", ws)
#print("ps= ", ps)
#print("val= ", val)
#print("node[cell]= ", node[cell])
#print("bb= ", bb)
#print("F= ", F)
print("uh= ", uh)

uh=  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  3.38813179e-20  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  2.52227589e-20  0.00000000e+00
  0.00000000e+00  0.00000000e+00  6.77626358e-21  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.54152159e-04
  4.81480675e-04 -8.54152159e-04  8.54152159e-04 -8.54152159e-04
 -4.81480675e-04]


In [62]:
space = LagrangeFiniteElementSpace(mesh, p=1)
bc = DirichletBC(space, pde.dirichlet, threshold=pde.is_dirichlet_boundary)
uh = space.function(dim=2)
F = space.source_vector(pde.source, dim=2)
#A, F = bc.apply(A, F, uh)

B = space.stiff_matrix(c=2*pde.mu)
C = space.linear_elasticity_matrix(pde.lam, pde.mu)

run serial_construct_matrix with time: 0.0017953920032596216
